In [4]:
import React, { useState, useEffect } from 'react';
import { LineChart, Line, AreaChart, Area, BarChart, Bar, XAxis, YAxis, CartesianGrid, Tooltip, Legend, ResponsiveContainer, ReferenceLine, ComposedChart } from 'recharts';

const FRED_API_KEY = '6dcc7a0d790cdcc28c1f751420ee9d27';
const OCEAN_BLUE = '#0066CC';
const DEEP_ORANGE = '#FF6B35';
const NEON_CAROLINA_BLUE = '#00C5FF';
const NEON_MAGENTA = '#FF00FF';
const MED_LIGHT_GRAY = '#999999';

// Custom axis styling
const axisStyle = {
  fontSize: 11,
  fill: MED_LIGHT_GRAY,
  fontFamily: 'system-ui, -apple-system, sans-serif'
};

const Watermark = () => (
  <div style={{
    position: 'absolute',
    bottom: '10px',
    right: '20px',
    color: OCEAN_BLUE,
    fontSize: '11px',
    fontWeight: 600,
    opacity: 0.6,
    fontFamily: 'system-ui, -apple-system, sans-serif',
    letterSpacing: '0.5px'
  }}>
    MACRO, ILLUMINATED.
  </div>
);

const ChartContainer = ({ title, children, description }) => (
  <div style={{ 
    marginBottom: '60px', 
    padding: '20px', 
    backgroundColor: '#fff',
    border: `1px solid ${MED_LIGHT_GRAY}`,
    borderRadius: '4px',
    position: 'relative'
  }}>
    <h3 style={{ 
      margin: '0 0 5px 0', 
      fontSize: '16px', 
      fontWeight: 600,
      color: '#333',
      fontFamily: 'system-ui, -apple-system, sans-serif'
    }}>
      {title}
    </h3>
    {description && (
      <p style={{ 
        margin: '0 0 20px 0', 
        fontSize: '13px', 
        color: '#666',
        fontFamily: 'system-ui, -apple-system, sans-serif',
        lineHeight: 1.4
      }}>
        {description}
      </p>
    )}
    <div style={{ position: 'relative', height: '400px' }}>
      {children}
      <Watermark />
    </div>
  </div>
);

const fetchFREDData = async (seriesId, startDate = '1995-01-01') => {
  try {
    const url = `https://api.stlouisfed.org/fred/series/observations?series_id=${seriesId}&api_key=${FRED_API_KEY}&file_type=json&observation_start=${startDate}`;
    const response = await fetch(url);
    const data = await response.json();
    return data.observations?.map(obs => ({
      date: obs.date,
      value: obs.value !== '.' ? parseFloat(obs.value) : null
    })).filter(d => d.value !== null) || [];
  } catch (error) {
    console.error(`Error fetching ${seriesId}:`, error);
    return [];
  }
};

const InflationDashboard = () => {
  const [cpiData, setCpiData] = useState([]);
  const [pceData, setPceData] = useState([]);
  const [disposableIncomeData, setDisposableIncomeData] = useState([]);
  const [supplyChainData, setSupplyChainData] = useState([]);
  const [energyData, setEnergyData] = useState([]);
  const [goodsServicesData, setGoodsServicesData] = useState([]);
  const [expectationsData, setExpectationsData] = useState([]);
  const [durableGoodsData, setDurableGoodsData] = useState([]);
  const [shelterData, setShelterData] = useState([]);
  const [wageData, setWageData] = useState([]);
  const [fedFundsData, setFedFundsData] = useState([]);
  const [debtData, setDebtData] = useState([]);
  const [tradeData, setTradeData] = useState([]);
  const [commodityData, setCommodityData] = useState([]);
  const [laborForceData, setLaborForceData] = useState([]);
  const [loading, setLoading] = useState(true);

  useEffect(() => {
    const loadData = async () => {
      setLoading(true);
      
      // Chart 1: CPI & Core CPI
      const [cpi, coreCPI] = await Promise.all([
        fetchFREDData('CPIAUCSL', '1990-01-01'),
        fetchFREDData('CPILFESL', '1990-01-01')
      ]);
      
      const cpiMerged = cpi.map((item, idx) => ({
        date: item.date,
        cpi: ((item.value / cpi[idx < 12 ? 0 : idx - 12].value - 1) * 100).toFixed(2),
        coreCPI: coreCPI[idx] ? ((coreCPI[idx].value / coreCPI[idx < 12 ? 0 : idx - 12].value - 1) * 100).toFixed(2) : null
      })).filter(d => d.cpi && d.coreCPI);
      setCpiData(cpiMerged);

      // Chart 2: Disposable Income
      const dpi = await fetchFREDData('DSPIC96', '1990-01-01');
      const dpiIndexed = dpi.map((item, idx) => ({
        date: item.date,
        value: ((item.value / dpi[0].value) * 100).toFixed(1)
      }));
      setDisposableIncomeData(dpiIndexed);

      // Chart 3 & 5: Supply Chain combined with CPI
      const gscpi = await fetchFREDData('GSCPI', '2000-01-01');
      const cpiForSupply = cpi.filter(d => d.date >= '2000-01-01');
      const supplyMerged = gscpi.map(item => {
        const matchingCPI = cpiForSupply.find(c => c.date === item.date);
        const cpiYoY = matchingCPI ? ((matchingCPI.value / cpiForSupply.find(c => c.date === new Date(new Date(item.date).getTime() - 365*24*60*60*1000).toISOString().split('T')[0])?.value - 1) * 100) : null;
        return {
          date: item.date,
          gscpi: parseFloat(item.value),
          cpi: cpiYoY
        };
      }).filter(d => d.cpi);
      setSupplyChainData(supplyMerged);

      // Chart 4: Energy (WTI, Natural Gas, Food)
      const [wti, natgas, food] = await Promise.all([
        fetchFREDData('DCOILWTICO', '2000-01-01'),
        fetchFREDData('DHHNGSP', '2000-01-01'),
        fetchFREDData('PPIACO', '2000-01-01')
      ]);
      
      const energyMerged = wti.map((item, idx) => ({
        date: item.date,
        wti: parseFloat(item.value),
        natgas: natgas[idx] ? parseFloat(natgas[idx].value) : null,
        food: food[idx] ? parseFloat(food[idx].value) : null
      })).filter(d => d.wti && d.natgas && d.food);
      setEnergyData(energyMerged);

      // Chart 6: Goods vs Services CPI
      const [goods, services] = await Promise.all([
        fetchFREDData('CUSR0000SAC', '2000-01-01'),
        fetchFREDData('CUSR0000SAS', '2000-01-01')
      ]);
      
      const goodsServicesMerged = goods.map((item, idx) => ({
        date: item.date,
        goods: idx >= 12 ? ((item.value / goods[idx - 12].value - 1) * 100).toFixed(2) : null,
        services: services[idx] && idx >= 12 ? ((services[idx].value / services[idx - 12].value - 1) * 100).toFixed(2) : null
      })).filter(d => d.goods && d.services);
      setGoodsServicesData(goodsServicesMerged);

      // Chart 7: Inflation Expectations (Michigan vs Breakeven)
      const [michigan, breakeven] = await Promise.all([
        fetchFREDData('MICH', '2000-01-01'),
        fetchFREDData('T5YIE', '2000-01-01')
      ]);
      
      const expectMerged = michigan.map(item => {
        const matchingBE = breakeven.find(b => b.date === item.date);
        return {
          date: item.date,
          michigan: parseFloat(item.value),
          breakeven: matchingBE ? parseFloat(matchingBE.value) : null
        };
      }).filter(d => d.breakeven);
      setExpectationsData(expectMerged);

      // Chart 8: Durable Goods CPI
      const durables = await fetchFREDData('CUSR0000SAD', '2000-01-01');
      const durablesYoY = durables.map((item, idx) => ({
        date: item.date,
        value: idx >= 12 ? ((item.value / durables[idx - 12].value - 1) * 100).toFixed(2) : null
      })).filter(d => d.value !== null);
      setDurableGoodsData(durablesYoY);

      // Chart 9: Shelter (CPI vs Zillow rent)
      const [cpiShelter, rentIndex] = await Promise.all([
        fetchFREDData('CUSR0000SAH', '2000-01-01'),
        fetchFREDData('ZRIMUS', '2015-01-01')
      ]);
      
      const shelterYoY = cpiShelter.map((item, idx) => ({
        date: item.date,
        cpiShelter: idx >= 12 ? ((item.value / cpiShelter[idx - 12].value - 1) * 100).toFixed(2) : null
      })).filter(d => d.cpiShelter !== null);
      
      const rentYoY = rentIndex.map((item, idx) => ({
        date: item.date,
        zillow: idx >= 12 ? ((item.value / rentIndex[idx - 12].value - 1) * 100).toFixed(2) : null
      })).filter(d => d.zillow !== null);
      
      const shelterMerged = shelterYoY.map(item => {
        const matchingRent = rentYoY.find(r => r.date === item.date);
        return {
          date: item.date,
          cpiShelter: item.cpiShelter,
          zillow: matchingRent?.zillow || null
        };
      });
      setShelterData(shelterMerged);

      // Chart 10: Wage Growth
      const wages = await fetchFREDData('CES0500000003', '1990-01-01');
      const wagesYoY = wages.map((item, idx) => ({
        date: item.date,
        value: idx >= 12 ? ((item.value / wages[idx - 12].value - 1) * 100).toFixed(2) : null
      })).filter(d => d.value !== null);
      setWageData(wagesYoY);

      // Chart 11: Fed Funds Rate
      const [ffr, realFFR] = await Promise.all([
        fetchFREDData('FEDFUNDS', '1980-01-01'),
        fetchFREDData('REAINTRATREARAT1YE', '1980-01-01')
      ]);
      
      const ffrMerged = ffr.map(item => {
        const matchingReal = realFFR.find(r => r.date === item.date);
        return {
          date: item.date,
          nominal: parseFloat(item.value),
          real: matchingReal ? parseFloat(matchingReal.value) : null
        };
      });
      setFedFundsData(ffrMerged);

      // Chart 12: Federal Debt
      const [debt, gdp, interest] = await Promise.all([
        fetchFREDData('GFDEBTN', '1990-01-01'),
        fetchFREDData('GDP', '1990-01-01'),
        fetchFREDData('A091RC1Q027SBEA', '1990-01-01')
      ]);
      
      const debtMerged = debt.map(item => {
        const matchingGDP = gdp.find(g => g.date === item.date);
        const matchingInt = interest.find(i => i.date === item.date);
        return {
          date: item.date,
          debtToGDP: matchingGDP ? ((item.value / matchingGDP.value) * 100).toFixed(1) : null,
          interestExp: matchingInt ? parseFloat(matchingInt.value) : null
        };
      }).filter(d => d.debtToGDP && d.interestExp);
      setDebtData(debtMerged);

      // Chart 13: Trade flows (Import shares)
      const [chinaImports, totalImports] = await Promise.all([
        fetchFREDData('IR4Q', '2000-01-01'),
        fetchFREDData('IMPGS', '2000-01-01')
      ]);
      
      const tradeMerged = chinaImports.map(item => {
        const matchingTotal = totalImports.find(t => t.date === item.date);
        return {
          date: item.date,
          chinaShare: matchingTotal ? ((item.value / matchingTotal.value) * 100).toFixed(1) : null
        };
      }).filter(d => d.chinaShare);
      setTradeData(tradeMerged);

      // Chart 14: Energy Transition (Copper, Lithium proxy)
      const [copper, palladium] = await Promise.all([
        fetchFREDData('PCOPPUSDM', '2000-01-01'),
        fetchFREDData('PPALLUSDM', '2000-01-01')
      ]);
      
      const commodMerged = copper.map((item, idx) => ({
        date: item.date,
        copper: parseFloat(item.value),
        palladium: palladium[idx] ? parseFloat(palladium[idx].value) : null
      })).filter(d => d.palladium);
      setCommodityData(commodMerged);

      // Chart 15: Labor Force Participation
      const [lfpr, primeAge] = await Promise.all([
        fetchFREDData('CIVPART', '1990-01-01'),
        fetchFREDData('LNS11300060', '1990-01-01')
      ]);
      
      const lfrMerged = lfpr.map((item, idx) => ({
        date: item.date,
        total: parseFloat(item.value),
        primeAge: primeAge[idx] ? parseFloat(primeAge[idx].value) : null
      })).filter(d => d.primeAge);
      setLaborForceData(lfrMerged);

      setLoading(false);
    };

    loadData();
  }, []);

  if (loading) {
    return (
      <div style={{ 
        padding: '40px', 
        textAlign: 'center',
        fontFamily: 'system-ui, -apple-system, sans-serif',
        fontSize: '16px',
        color: '#666'
      }}>
        Loading data from FRED...
      </div>
    );
  }

  return (
    <div style={{ 
      padding: '20px', 
      maxWidth: '1400px', 
      margin: '0 auto',
      fontFamily: 'system-ui, -apple-system, sans-serif'
    }}>
      <h1 style={{ 
        fontSize: '32px', 
        fontWeight: 700, 
        marginBottom: '10px',
        color: OCEAN_BLUE
      }}>
        The Inflationary Backdrop
      </h1>
      <h2 style={{ 
        fontSize: '18px', 
        fontWeight: 400, 
        marginBottom: '40px',
        color: '#666',
        fontStyle: 'italic'
      }}>
        Lighthouse Macro | Macro, Illuminated.
      </h2>

      {/* Chart 1 */}
      <ChartContainer 
        title="Chart 1: Inflation Re-emerges as the Constraint"
        description="U.S. CPI inflation surged above 9% in 2022—the highest since the early 1980s. Core CPI (excluding food and energy) also moved far above the Fed's 2% target, ending decades of subdued price pressure."
      >
        <ResponsiveContainer width="100%" height="100%">
          <LineChart data={cpiData} margin={{ top: 10, right: 30, left: 10, bottom: 20 }}>
            <XAxis 
              dataKey="date" 
              tick={axisStyle}
              tickFormatter={(date) => new Date(date).getFullYear()}
              interval="preserveStartEnd"
            />
            <YAxis 
              yAxisId="left"
              orientation="right"
              tick={axisStyle}
              label={{ value: 'Year-over-Year %', angle: 90, position: 'insideRight', style: axisStyle }}
            />
            <Tooltip 
              contentStyle={{ 
                backgroundColor: '#fff', 
                border: `1px solid ${MED_LIGHT_GRAY}`,
                borderRadius: '4px',
                fontSize: '12px'
              }}
              labelFormatter={(date) => new Date(date).toLocaleDateString()}
              formatter={(value) => [`${value}%`, '']}
            />
            <ReferenceLine yAxisId="left" y={2} stroke={MED_LIGHT_GRAY} strokeDasharray="3 3" />
            <Legend wrapperStyle={{ fontSize: '12px' }} />
            <Line 
              yAxisId="left"
              type="monotone" 
              dataKey="cpi" 
              stroke={OCEAN_BLUE} 
              strokeWidth={2.5}
              dot={false}
              name="CPI (All Items)"
            />
            <Line 
              yAxisId="left"
              type="monotone" 
              dataKey="coreCPI" 
              stroke={DEEP_ORANGE} 
              strokeWidth={2.5}
              dot={false}
              name="Core CPI"
            />
          </LineChart>
        </ResponsiveContainer>
      </ChartContainer>

      {/* Chart 2 */}
      <ChartContainer 
        title="Chart 2: Demand Overshoot—Disposable Personal Income"
        description="Fiscal transfers during COVID pushed disposable income far above trend. Households accumulated excess savings that later fueled spending, creating classic demand-pull inflation as restrictions lifted."
      >
        <ResponsiveContainer width="100%" height="100%">
          <AreaChart data={disposableIncomeData} margin={{ top: 10, right: 30, left: 10, bottom: 20 }}>
            <defs>
              <linearGradient id="colorIncome" x1="0" y1="0" x2="0" y2="1">
                <stop offset="5%" stopColor={OCEAN_BLUE} stopOpacity={0.3}/>
                <stop offset="95%" stopColor={OCEAN_BLUE} stopOpacity={0}/>
              </linearGradient>
            </defs>
            <XAxis 
              dataKey="date" 
              tick={axisStyle}
              tickFormatter={(date) => new Date(date).getFullYear()}
              interval="preserveStartEnd"
            />
            <YAxis 
              orientation="right"
              tick={axisStyle}
              label={{ value: 'Index (1990 = 100)', angle: 90, position: 'insideRight', style: axisStyle }}
            />
            <Tooltip 
              contentStyle={{ 
                backgroundColor: '#fff', 
                border: `1px solid ${MED_LIGHT_GRAY}`,
                borderRadius: '4px',
                fontSize: '12px'
              }}
              labelFormatter={(date) => new Date(date).toLocaleDateString()}
            />
            <Legend wrapperStyle={{ fontSize: '12px' }} />
            <Area 
              type="monotone" 
              dataKey="value" 
              stroke={OCEAN_BLUE} 
              strokeWidth={2.5}
              fillOpacity={1}
              fill="url(#colorIncome)"
              name="Real Disposable Income"
            />
          </AreaChart>
        </ResponsiveContainer>
      </ChartContainer>

      {/* Chart 3 */}
      <ChartContainer 
        title="Chart 3: Supply Under Strain—Global Supply Chain Pressure"
        description="Global supply chain disruptions spiked during the pandemic. Ports clogged, container costs soared, and delivery times stretched. These constraints amplified goods inflation as demand recovered."
      >
        <ResponsiveContainer width="100%" height="100%">
          <LineChart data={supplyChainData} margin={{ top: 10, right: 30, left: 10, bottom: 20 }}>
            <XAxis 
              dataKey="date" 
              tick={axisStyle}
              tickFormatter={(date) => new Date(date).getFullYear()}
              interval="preserveStartEnd"
            />
            <YAxis 
              orientation="right"
              tick={axisStyle}
              label={{ value: 'Index (Std Dev)', angle: 90, position: 'insideRight', style: axisStyle }}
            />
            <Tooltip 
              contentStyle={{ 
                backgroundColor: '#fff', 
                border: `1px solid ${MED_LIGHT_GRAY}`,
                borderRadius: '4px',
                fontSize: '12px'
              }}
              labelFormatter={(date) => new Date(date).toLocaleDateString()}
            />
            <Legend wrapperStyle={{ fontSize: '12px' }} />
            <Line 
              type="monotone" 
              dataKey="gscpi" 
              stroke={DEEP_ORANGE} 
              strokeWidth={2.5}
              dot={false}
              name="Global Supply Chain Pressure Index"
            />
          </LineChart>
        </ResponsiveContainer>
      </ChartContainer>

      {/* Chart 4 */}
      <ChartContainer 
        title="Chart 4: Energy and Commodity Shock"
        description="Russia's invasion of Ukraine in 2022 created a massive energy shock. WTI crude and natural gas prices spiked, broadening and prolonging inflation pressures globally."
      >
        <ResponsiveContainer width="100%" height="100%">
          <ComposedChart data={energyData} margin={{ top: 10, right: 30, left: 10, bottom: 20 }}>
            <XAxis 
              dataKey="date" 
              tick={axisStyle}
              tickFormatter={(date) => new Date(date).getFullYear()}
              interval="preserveStartEnd"
            />
            <YAxis 
              yAxisId="left"
              orientation="right"
              tick={axisStyle}
              label={{ value: 'WTI Crude ($/barrel)', angle: 90, position: 'insideRight', style: axisStyle }}
            />
            <YAxis 
              yAxisId="right"
              orientation="left"
              tick={axisStyle}
              label={{ value: 'Natural Gas ($/MMBtu)', angle: -90, position: 'insideLeft', style: axisStyle }}
            />
            <Tooltip 
              contentStyle={{ 
                backgroundColor: '#fff', 
                border: `1px solid ${MED_LIGHT_GRAY}`,
                borderRadius: '4px',
                fontSize: '12px'
              }}
              labelFormatter={(date) => new Date(date).toLocaleDateString()}
              formatter={(value) => [value.toFixed(2), '']}
            />
            <Legend wrapperStyle={{ fontSize: '12px' }} />
            <Line 
              yAxisId="left"
              type="monotone" 
              dataKey="wti" 
              stroke={OCEAN_BLUE} 
              strokeWidth={2.5}
              dot={false}
              name="WTI Crude Oil"
            />
            <Line 
              yAxisId="right"
              type="monotone" 
              dataKey="natgas" 
              stroke={DEEP_ORANGE} 
              strokeWidth={2.5}
              dot={false}
              name="Natural Gas"
            />
          </ComposedChart>
        </ResponsiveContainer>
      </ChartContainer>

      {/* Chart 5 */}
      <ChartContainer 
        title="Chart 5: Supply vs Demand—GSCPI and CPI"
        description="The Global Supply Chain Pressure Index tracked CPI inflation closely on the way up. But when supply normalized, inflation remained elevated—signaling demand and services, not just supply, were driving persistence."
      >
        <ResponsiveContainer width="100%" height="100%">
          <ComposedChart data={supplyChainData} margin={{ top: 10, right: 30, left: 10, bottom: 20 }}>
            <XAxis 
              dataKey="date" 
              tick={axisStyle}
              tickFormatter={(date) => new Date(date).getFullYear()}
              interval="preserveStartEnd"
            />
            <YAxis 
              yAxisId="left"
              orientation="right"
              tick={axisStyle}
              label={{ value: 'GSCPI (Std Dev)', angle: 90, position: 'insideRight', style: axisStyle }}
            />
            <YAxis 
              yAxisId="right"
              orientation="left"
              tick={axisStyle}
              label={{ value: 'CPI YoY %', angle: -90, position: 'insideLeft', style: axisStyle }}
            />
            <Tooltip 
              contentStyle={{ 
                backgroundColor: '#fff', 
                border: `1px solid ${MED_LIGHT_GRAY}`,
                borderRadius: '4px',
                fontSize: '12px'
              }}
              labelFormatter={(date) => new Date(date).toLocaleDateString()}
              formatter={(value, name) => [name === 'CPI' ? `${value}%` : value.toFixed(2), '']}
            />
            <Legend wrapperStyle={{ fontSize: '12px' }} />
            <Line 
              yAxisId="left"
              type="monotone" 
              dataKey="gscpi" 
              stroke={DEEP_ORANGE} 
              strokeWidth={2.5}
              dot={false}
              name="Supply Chain Pressure"
            />
            <Line 
              yAxisId="right"
              type="monotone" 
              dataKey="cpi" 
              stroke={OCEAN_BLUE} 
              strokeWidth={2.5}
              dot={false}
              name="CPI Inflation"
            />
          </ComposedChart>
        </ResponsiveContainer>
      </ChartContainer>

      {/* Chart 6 */}
      <ChartContainer 
        title="Chart 6: Goods vs Services Inflation—The Last Mile Problem"
        description="Goods inflation surged early, then reversed as supply chains healed. Services inflation never corrected the same way. Labor-intensive categories like rent and healthcare remain sticky—this is the 'last mile' challenge."
      >
        <ResponsiveContainer width="100%" height="100%">
          <LineChart data={goodsServicesData} margin={{ top: 10, right: 30, left: 10, bottom: 20 }}>
            <XAxis 
              dataKey="date" 
              tick={axisStyle}
              tickFormatter={(date) => new Date(date).getFullYear()}
              interval="preserveStartEnd"
            />
            <YAxis 
              orientation="right"
              tick={axisStyle}
              label={{ value: 'Year-over-Year %', angle: 90, position: 'insideRight', style: axisStyle }}
            />
            <Tooltip 
              contentStyle={{ 
                backgroundColor: '#fff', 
                border: `1px solid ${MED_LIGHT_GRAY}`,
                borderRadius: '4px',
                fontSize: '12px'
              }}
              labelFormatter={(date) => new Date(date).toLocaleDateString()}
              formatter={(value) => [`${value}%`, '']}
            />
            <ReferenceLine y={2} stroke={MED_LIGHT_GRAY} strokeDasharray="3 3" />
            <Legend wrapperStyle={{ fontSize: '12px' }} />
            <Line 
              type="monotone" 
              dataKey="goods" 
              stroke={NEON_CAROLINA_BLUE} 
              strokeWidth={2.5}
              dot={false}
              name="Goods Inflation"
            />
            <Line 
              type="monotone" 
              dataKey="services" 
              stroke={NEON_MAGENTA} 
              strokeWidth={2.5}
              dot={false}
              name="Services Inflation"
            />
          </LineChart>
        </ResponsiveContainer>
      </ChartContainer>

      {/* Chart 7 */}
      <ChartContainer 
        title="Chart 7: Inflation Expectations—Survey vs Market"
        description="Survey-based expectations (Michigan) rose faster than market-based measures (5Y breakeven). The divergence highlights a credibility challenge: households expect higher inflation while markets remain relatively anchored."
      >
        <ResponsiveContainer width="100%" height="100%">
          <LineChart data={expectationsData} margin={{ top: 10, right: 30, left: 10, bottom: 20 }}>
            <XAxis 
              dataKey="date" 
              tick={axisStyle}
              tickFormatter={(date) => new Date(date).getFullYear()}
              interval="preserveStartEnd"
            />
            <YAxis 
              orientation="right"
              tick={axisStyle}
              label={{ value: 'Expected Inflation (%)', angle: 90, position: 'insideRight', style: axisStyle }}
            />
            <Tooltip 
              contentStyle={{ 
                backgroundColor: '#fff', 
                border: `1px solid ${MED_LIGHT_GRAY}`,
                borderRadius: '4px',
                fontSize: '12px'
              }}
              labelFormatter={(date) => new Date(date).toLocaleDateString()}
              formatter={(value) => [`${value}%`, '']}
            />
            <ReferenceLine y={2} stroke={MED_LIGHT_GRAY} strokeDasharray="3 3" />
            <Legend wrapperStyle={{ fontSize: '12px' }} />
            <Line 
              type="monotone" 
              dataKey="michigan" 
              stroke={DEEP_ORANGE} 
              strokeWidth={2.5}
              dot={false}
              name="Michigan Survey"
            />
            <Line 
              type="monotone" 
              dataKey="breakeven" 
              stroke={OCEAN_BLUE} 
              strokeWidth={2.5}
              dot={false}
              name="5Y Breakeven"
            />
          </LineChart>
        </ResponsiveContainer>
      </ChartContainer>

      {/* Chart 8 */}
      <ChartContainer 
        title="Chart 8: Goods Reversal—Durable Goods Deflation"
        description="Durable goods prices surged during the pandemic, then reversed sharply. Used car prices, furniture, and electronics have all corrected. Goods inflation is no longer the primary driver."
      >
        <ResponsiveContainer width="100%" height="100%">
          <AreaChart data={durableGoodsData} margin={{ top: 10, right: 30, left: 10, bottom: 20 }}>
            <defs>
              <linearGradient id="colorDurables" x1="0" y1="0" x2="0" y2="1">
                <stop offset="5%" stopColor={NEON_CAROLINA_BLUE} stopOpacity={0.3}/>
                <stop offset="95%" stopColor={NEON_CAROLINA_BLUE} stopOpacity={0}/>
              </linearGradient>
            </defs>
            <XAxis 
              dataKey="date" 
              tick={axisStyle}
              tickFormatter={(date) => new Date(date).getFullYear()}
              interval="preserveStartEnd"
            />
            <YAxis 
              orientation="right"
              tick={axisStyle}
              label={{ value: 'Year-over-Year %', angle: 90, position: 'insideRight', style: axisStyle }}
            />
            <Tooltip 
              contentStyle={{ 
                backgroundColor: '#fff', 
                border: `1px solid ${MED_LIGHT_GRAY}`,
                borderRadius: '4px',
                fontSize: '12px'
              }}
              labelFormatter={(date) => new Date(date).toLocaleDateString()}
              formatter={(value) => [`${value}%`, '']}
            />
            <ReferenceLine y={0} stroke={MED_LIGHT_GRAY} strokeDasharray="3 3" />
            <Legend wrapperStyle={{ fontSize: '12px' }} />
            <Area 
              type="monotone" 
              dataKey="value" 
              stroke={NEON_CAROLINA_BLUE} 
              strokeWidth={2.5}
              fillOpacity={1}
              fill="url(#colorDurables)"
              name="Durable Goods CPI"
            />
          </AreaChart>
        </ResponsiveContainer>
      </ChartContainer>

      {/* Chart 9 */}
      <ChartContainer 
        title="Chart 9: Shelter Stickiness—CPI vs Market Rents"
        description="Private rental data (Zillow) shows rents peaked months ago. But official CPI shelter measures remain elevated due to measurement lags. This keeps headline inflation sticky even as real-time rents cool."
      >
        <ResponsiveContainer width="100%" height="100%">
          <LineChart data={shelterData} margin={{ top: 10, right: 30, left: 10, bottom: 20 }}>
            <XAxis 
              dataKey="date" 
              tick={axisStyle}
              tickFormatter={(date) => new Date(date).getFullYear()}
              interval="preserveStartEnd"
            />
            <YAxis 
              orientation="right"
              tick={axisStyle}
              label={{ value: 'Year-over-Year %', angle: 90, position: 'insideRight', style: axisStyle }}
            />
            <Tooltip 
              contentStyle={{ 
                backgroundColor: '#fff', 
                border: `1px solid ${MED_LIGHT_GRAY}`,
                borderRadius: '4px',
                fontSize: '12px'
              }}
              labelFormatter={(date) => new Date(date).toLocaleDateString()}
              formatter={(value) => value ? [`${value}%`, ''] : ['N/A', '']}
            />
            <ReferenceLine y={2} stroke={MED_LIGHT_GRAY} strokeDasharray="3 3" />
            <Legend wrapperStyle={{ fontSize: '12px' }} />
            <Line 
              type="monotone" 
              dataKey="cpiShelter" 
              stroke={OCEAN_BLUE} 
              strokeWidth={2.5}
              dot={false}
              name="CPI Shelter"
            />
            <Line 
              type="monotone" 
              dataKey="zillow" 
              stroke={DEEP_ORANGE} 
              strokeWidth={2.5}
              dot={false}
              name="Zillow Rent Index"
              connectNulls
            />
          </LineChart>
        </ResponsiveContainer>
      </ChartContainer>

      {/* Chart 10 */}
      <ChartContainer 
        title="Chart 10: Wage Growth—The Services Inflation Channel"
        description="Wage growth has slowed from pandemic peaks but remains elevated compared to the 2010s. As long as wages run above 2%, it's difficult to fully break services inflation. This is the feedback loop policymakers watch closely."
      >
        <ResponsiveContainer width="100%" height="100%">
          <AreaChart data={wageData} margin={{ top: 10, right: 30, left: 10, bottom: 20 }}>
            <defs>
              <linearGradient id="colorWages" x1="0" y1="0" x2="0" y2="1">
                <stop offset="5%" stopColor={OCEAN_BLUE} stopOpacity={0.3}/>
                <stop offset="95%" stopColor={OCEAN_BLUE} stopOpacity={0}/>
              </linearGradient>
            </defs>
            <XAxis 
              dataKey="date" 
              tick={axisStyle}
              tickFormatter={(date) => new Date(date).getFullYear()}
              interval="preserveStartEnd"
            />
            <YAxis 
              orientation="right"
              tick={axisStyle}
              label={{ value: 'Year-over-Year %', angle: 90, position: 'insideRight', style: axisStyle }}
            />
            <Tooltip 
              contentStyle={{ 
                backgroundColor: '#fff', 
                border: `1px solid ${MED_LIGHT_GRAY}`,
                borderRadius: '4px',
                fontSize: '12px'
              }}
              labelFormatter={(date) => new Date(date).toLocaleDateString()}
              formatter={(value) => [`${value}%`, '']}
            />
            <ReferenceLine y={2} stroke={MED_LIGHT_GRAY} strokeDasharray="3 3" />
            <Legend wrapperStyle={{ fontSize: '12px' }} />
            <Area 
              type="monotone" 
              dataKey="value" 
              stroke={OCEAN_BLUE} 
              strokeWidth={2.5}
              fillOpacity={1}
              fill="url(#colorWages)"
              name="Average Hourly Earnings"
            />
          </AreaChart>
        </ResponsiveContainer>
      </ChartContainer>

      {/* Chart 11 */}
      <ChartContainer 
        title="Chart 11: Policy Response—Fed Funds Rate Hiking Cycle"
        description="The Fed hiked faster than any time since the Volcker era. Real policy rates finally turned positive. But lags mean the full effect isn't yet visible. The Fed must balance easing too soon (inflation revival) vs staying too tight (breaking growth)."
      >
        <ResponsiveContainer width="100%" height="100%">
          <ComposedChart data={fedFundsData} margin={{ top: 10, right: 30, left: 10, bottom: 20 }}>
            <XAxis 
              dataKey="date" 
              tick={axisStyle}
              tickFormatter={(date) => new Date(date).getFullYear()}
              interval="preserveStartEnd"
            />
            <YAxis 
              orientation="right"
              tick={axisStyle}
              label={{ value: 'Interest Rate (%)', angle: 90, position: 'insideRight', style: axisStyle }}
            />
            <Tooltip 
              contentStyle={{ 
                backgroundColor: '#fff', 
                border: `1px solid ${MED_LIGHT_GRAY}`,
                borderRadius: '4px',
                fontSize: '12px'
              }}
              labelFormatter={(date) => new Date(date).toLocaleDateString()}
              formatter={(value) => value !== null ? [`${value.toFixed(2)}%`, ''] : ['N/A', '']}
            />
            <ReferenceLine y={0} stroke={MED_LIGHT_GRAY} strokeDasharray="3 3" />
            <Legend wrapperStyle={{ fontSize: '12px' }} />
            <Line 
              type="monotone" 
              dataKey="nominal" 
              stroke={OCEAN_BLUE} 
              strokeWidth={2.5}
              dot={false}
              name="Nominal Fed Funds"
            />
            <Line 
              type="monotone" 
              dataKey="real" 
              stroke={DEEP_ORANGE} 
              strokeWidth={2.5}
              dot={false}
              name="Real Fed Funds"
              connectNulls
            />
          </ComposedChart>
        </ResponsiveContainer>
      </ChartContainer>

      {/* Chart 12 */}
      <ChartContainer 
        title="Chart 12: Fiscal Math—Debt and Interest Expense"
        description="Federal debt remains elevated as a share of GDP. With higher rates, interest expense is climbing sharply. Fiscal sustainability is once again a macro constraint—something that didn't matter in a zero-rate world."
      >
        <ResponsiveContainer width="100%" height="100%">
          <ComposedChart data={debtData} margin={{ top: 10, right: 30, left: 10, bottom: 20 }}>
            <XAxis 
              dataKey="date" 
              tick={axisStyle}
              tickFormatter={(date) => new Date(date).getFullYear()}
              interval="preserveStartEnd"
            />
            <YAxis 
              yAxisId="left"
              orientation="right"
              tick={axisStyle}
              label={{ value: 'Debt/GDP (%)', angle: 90, position: 'insideRight', style: axisStyle }}
            />
            <YAxis 
              yAxisId="right"
              orientation="left"
              tick={axisStyle}
              label={{ value: 'Interest Exp ($ Bil)', angle: -90, position: 'insideLeft', style: axisStyle }}
            />
            <Tooltip 
              contentStyle={{ 
                backgroundColor: '#fff', 
                border: `1px solid ${MED_LIGHT_GRAY}`,
                borderRadius: '4px',
                fontSize: '12px'
              }}
              labelFormatter={(date) => new Date(date).toLocaleDateString()}
              formatter={(value, name) => [name === 'Debt to GDP' ? `${value}%` : `$${value}B`, '']}
            />
            <Legend wrapperStyle={{ fontSize: '12px' }} />
            <Area 
              yAxisId="left"
              type="monotone" 
              dataKey="debtToGDP" 
              stroke={OCEAN_BLUE} 
              strokeWidth={2}
              fill={OCEAN_BLUE}
              fillOpacity={0.2}
              name="Debt to GDP"
            />
            <Line 
              yAxisId="right"
              type="monotone" 
              dataKey="interestExp" 
              stroke={DEEP_ORANGE} 
              strokeWidth={2.5}
              dot={false}
              name="Interest Expense"
            />
          </ComposedChart>
        </ResponsiveContainer>
      </ChartContainer>

      {/* Chart 13 */}
      <ChartContainer 
        title="Chart 13: Deglobalization—Shifting Trade Flows"
        description="China's share of U.S. imports is declining as firms de-risk supply chains. Mexico and Vietnam are gaining. This shift makes supply more resilient but adds near-term costs—a modest inflationary force."
      >
        <ResponsiveContainer width="100%" height="100%">
          <AreaChart data={tradeData} margin={{ top: 10, right: 30, left: 10, bottom: 20 }}>
            <defs>
              <linearGradient id="colorTrade" x1="0" y1="0" x2="0" y2="1">
                <stop offset="5%" stopColor={DEEP_ORANGE} stopOpacity={0.3}/>
                <stop offset="95%" stopColor={DEEP_ORANGE} stopOpacity={0}/>
              </linearGradient>
            </defs>
            <XAxis 
              dataKey="date" 
              tick={axisStyle}
              tickFormatter={(date) => new Date(date).getFullYear()}
              interval="preserveStartEnd"
            />
            <YAxis 
              orientation="right"
              tick={axisStyle}
              label={{ value: 'Share of Total Imports (%)', angle: 90, position: 'insideRight', style: axisStyle }}
            />
            <Tooltip 
              contentStyle={{ 
                backgroundColor: '#fff', 
                border: `1px solid ${MED_LIGHT_GRAY}`,
                borderRadius: '4px',
                fontSize: '12px'
              }}
              labelFormatter={(date) => new Date(date).toLocaleDateString()}
              formatter={(value) => [`${value}%`, '']}
            />
            <Legend wrapperStyle={{ fontSize: '12px' }} />
            <Area 
              type="monotone" 
              dataKey="chinaShare" 
              stroke={DEEP_ORANGE} 
              strokeWidth={2.5}
              fillOpacity={1}
              fill="url(#colorTrade)"
              name="China Share of US Imports"
            />
          </AreaChart>
        </ResponsiveContainer>
      </ChartContainer>

      {/* Chart 14 */}
      <ChartContainer 
        title="Chart 14: Energy Transition—Commodity Intensity"
        description="The green transition requires massive upfront investment in metals like copper and palladium. This creates near-term inflation pressure before renewables scale and become disinflationary."
      >
        <ResponsiveContainer width="100%" height="100%">
          <LineChart data={commodityData} margin={{ top: 10, right: 30, left: 10, bottom: 20 }}>
            <XAxis 
              dataKey="date" 
              tick={axisStyle}
              tickFormatter={(date) => new Date(date).getFullYear()}
              interval="preserveStartEnd"
            />
            <YAxis 
              yAxisId="left"
              orientation="right"
              tick={axisStyle}
              label={{ value: 'Copper ($/MT)', angle: 90, position: 'insideRight', style: axisStyle }}
            />
            <YAxis 
              yAxisId="right"
              orientation="left"
              tick={axisStyle}
              label={{ value: 'Palladium ($/oz)', angle: -90, position: 'insideLeft', style: axisStyle }}
            />
            <Tooltip 
              contentStyle={{ 
                backgroundColor: '#fff', 
                border: `1px solid ${MED_LIGHT_GRAY}`,
                borderRadius: '4px',
                fontSize: '12px'
              }}
              labelFormatter={(date) => new Date(date).toLocaleDateString()}
              formatter={(value) => [value.toFixed(0), '']}
            />
            <Legend wrapperStyle={{ fontSize: '12px' }} />
            <Line 
              yAxisId="left"
              type="monotone" 
              dataKey="copper" 
              stroke={DEEP_ORANGE} 
              strokeWidth={2.5}
              dot={false}
              name="Copper Price"
            />
            <Line 
              yAxisId="right"
              type="monotone" 
              dataKey="palladium" 
              stroke={NEON_CAROLINA_BLUE} 
              strokeWidth={2.5}
              dot={false}
              name="Palladium Price"
            />
          </LineChart>
        </ResponsiveContainer>
      </ChartContainer>

      {/* Chart 15 */}
      <ChartContainer 
        title="Chart 15: Demographics—Labor Force Participation"
        description="Prime-age participation has rebounded, but aging demographics dominate the long run. Labor force growth is slowing structurally, feeding directly into wage pressure and services inflation—a long-term headwind to disinflation."
      >
        <ResponsiveContainer width="100%" height="100%">
          <LineChart data={laborForceData} margin={{ top: 10, right: 30, left: 10, bottom: 20 }}>
            <XAxis 
              dataKey="date" 
              tick={axisStyle}
              tickFormatter={(date) => new Date(date).getFullYear()}
              interval="preserveStartEnd"
            />
            <YAxis 
              orientation="right"
              tick={axisStyle}
              domain={[58, 85]}
              label={{ value: 'Participation Rate (%)', angle: 90, position: 'insideRight', style: axisStyle }}
            />
            <Tooltip 
              contentStyle={{ 
                backgroundColor: '#fff', 
                border: `1px solid ${MED_LIGHT_GRAY}`,
                borderRadius: '4px',
                fontSize: '12px'
              }}
              labelFormatter={(date) => new Date(date).toLocaleDateString()}
              formatter={(value) => [`${value}%`, '']}
            />
            <Legend wrapperStyle={{ fontSize: '12px' }} />
            <Line 
              type="monotone" 
              dataKey="total" 
              stroke={OCEAN_BLUE} 
              strokeWidth={2.5}
              dot={false}
              name="Total LFPR"
            />
            <Line 
              type="monotone" 
              dataKey="primeAge" 
              stroke={NEON_MAGENTA} 
              strokeWidth={2.5}
              dot={false}
              name="Prime Age (25-54)"
            />
          </LineChart>
        </ResponsiveContainer>
      </ChartContainer>

      <div style={{ 
        marginTop: '60px', 
        padding: '20px', 
        backgroundColor: '#f5f5f5',
        border: `1px solid ${MED_LIGHT_GRAY}`,
        borderRadius: '4px'
      }}>
        <p style={{ 
          margin: 0, 
          fontSize: '12px', 
          color: '#666',
          lineHeight: 1.6
        }}>
          <strong>Data Sources:</strong> Federal Reserve Economic Data (FRED), Federal Reserve Bank of St. Louis, Bureau of Labor Statistics, Federal Reserve Bank of New York, Zillow Research.
          <br/><br/>
          <strong>Disclaimer:</strong> This report is for informational and educational purposes only. It is not financial advice. All analysis represents the views of Lighthouse Macro and should not be construed as investment recommendations.
        </p>
      </div>
    </div>
  );
};

export default InflationDashboard;

SyntaxError: invalid decimal literal (1870483339.py, line 39)